Open question 7:

In [ ]:
## Detailed Architecture Flow

1) Object Storage (S3 Bucket):

    Purpose: Store uploaded images.

    Setup:

    -Create an S3 bucket (e.g., image-upload-bucket).

    -Configure S3 Event Notifications to send events to an SQS queue for each new image.

2) Batch Trigger (SQS Queue):

    Purpose: Buffer image events and trigger processing when 20 images accumulate.

    Setup:

    -Create a Standard SQS Queue (e.g., image-batch-queue).

    -Configure S3 Event Notifications to send events to this queue.

    -Use CloudWatch Alarm to monitor queue depth:

    -Metric: ApproximateNumberOfMessagesVisible

    -Alarm Condition: >= 20 images

    -Attach the alarm to an AWS Lambda function (Batch Processor) via EventBridge.

3) Batch Processor (Lambda 1):

    Purpose: Group 20 images into a batch and trigger processing.

    Setup:

    -Using the custom container image of this repo (i might use ECR to store the image)

    Logic:

    -Fetch messages from SQS (up to 20 per invocation).

    -Extract S3 object keys from messages.

    -Invoke the Processor Lambda with the batch of 100 keys.

    -Delete processed messages from SQS.

    -IAM Role: Access to SQS, Lambda invocation, and S3 read.

4) Processor (Lambda 2):

    Purpose: Process images, save results to DB, and notify users.

    Logic:

    Request 20 images from S3.

    Process images (This will require adding a image loader addition to the container).

    Write results to DynamoDB. (This will require adding writing to dynamo from the container)

    Publish success message to SNS.

    IAM Role: Access to S3, DynamoDB, and SNS.

5) Database (DynamoDB):

    Purpose: Store processed image metadata and batch info.

    Table Design:

        Table 1: ProcessedBatches (Batch-level metadata)

        batch_id

        image_count (e.g., 20)

        status (COMPLETED or ERROR)

        processed_at (timestamp)

        stats (the dictionnary of stats)

6) User Notification (SNS):

    Purpose: Alert end-users upon batch completion.

    Setup:

    -Create an SNS topic (e.g., batch-completion-topic).

    -Subscribe users (email/SMS/HTTP).

    -Processor Lambda publishes to this topic after DynamoDB write.
